In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# CSV 파일 불러오기
file_path1 = '/content/drive/MyDrive/2024-2학기_연구참여/data/Meteorological_2019_01.csv'
file_path2 = '/content/drive/MyDrive/2024-2학기_연구참여/data/서울시_대기질_2019_01.csv'
met_data = pd.read_csv(file_path1, usecols=lambda column: column != '지점', encoding='cp949')
air_quality_data = pd.read_csv(file_path2, usecols = ['측정일시', '측정소명', '오존(ppm)'], encoding='cp949')
air_quality_data = air_quality_data[['측정소명', '측정일시', '오존(ppm)']]
print(met_data.columns)
print(air_quality_data.columns)
print(air_quality_data['측정일시'])

Index(['지점명', '일시', '기온(°C)'], dtype='object')
Index(['측정소명', '측정일시', '오존(ppm)'], dtype='object')
0        201901312300
1        201901312300
2        201901312300
3        201901312300
4        201901312300
             ...     
18595    201901010000
18596    201901010000
18597    201901010000
18598    201901010000
18599    201901010000
Name: 측정일시, Length: 18600, dtype: int64


In [7]:
# 날짜 및 시간 데이터 처리
met_data['일시'] = pd.to_datetime(met_data['일시'])
air_quality_data['측정일시'] = pd.to_datetime(air_quality_data['측정일시'], format='%Y%m%d%H%M')
air_quality_data = air_quality_data.sort_values(by=['측정소명', '측정일시']).reset_index(drop=True)
#print(met_data['일시'])
#print(air_quality_data)

In [8]:
# 'Meteorological data' 측정 지역의 위도와 경도 정보
met_regions_data = {
    'region': ['강남', '서초', '강동', '송파', '강서', '양천', '도봉', '노원', '동대문', '중랑', '기상청', '마포', '서대문', '광진',
               '성북', '용산', '은평', '금천', '한강', '중구', '성동', '구로', '강북*', '남현', '관악', '영등포', '현충원'],
    'latitude': [37.49794, 37.47650, 37.53010, 37.51456, 37.55095, 37.51656, 37.66877, 37.65526, 37.57491, 37.59508, 37.56556,
                 37.56632, 37.57655, 37.53865, 37.58968, 37.53110, 37.60277, 37.45686, 37.52770, 37.56382, 37.56351, 37.49543,
                 37.646995, 37.46769, 37.47808, 37.52667, 37.50001],
    'longitude': [127.02761, 127.03250, 127.12587, 127.10662, 126.84953, 126.86658, 127.04744, 127.07712, 127.03963, 127.09368, 126.96691,
                  126.90131, 126.93784, 127.08237, 127.01667, 126.97960, 126.93040, 126.89576, 126.99818, 126.99760, 127.03722, 126.88740,
                  127.01371, 126.95818, 126.95143, 126.89636, 126.97422]
}

# DataFrame 생성
met_regions_df = pd.DataFrame(met_regions_data)

# 출력
print(met_regions_df)

# '서울시 대기질 데이터' 각 구의 위도와 경도 정보
air_regions_data = {
    'region': ['종로구', '용산구', '중구', '은평구', '서대문구', '마포구', '광진구', '노원구', '중랑구', '도봉구',
               '성북구', '성동구', '강북구', '동대문구', '동작구', '영등포구', '강서구', '관악구', '양천구', '구로구',
               '금천구', '강남구', '서초구', '강동구', '송파구'],
    'latitude': [37.573050, 37.531100, 37.563820, 37.602770, 37.576550, 37.566320, 37.538650, 37.655260, 37.595080, 37.668770,
                 37.589680, 37.563510, 37.646995, 37.574910, 37.512410, 37.526670, 37.550950, 37.478080, 37.516560, 37.495430,
                 37.456860, 37.497940, 37.476500, 37.530100, 37.514560],
    'longitude': [126.979340, 126.979600, 126.997600, 126.930400, 126.937840, 126.901310, 127.082370, 127.077120, 127.093680, 127.047440,
                  127.016670, 127.037220, 127.013710, 127.039630, 126.941040, 126.896360, 126.849530, 126.951430, 126.866580, 126.887400,
                  126.895760, 127.027610, 127.032500, 127.125870, 127.106620]
}

# DataFrame 생성
air_regions_df = pd.DataFrame(air_regions_data)

# 출력
print(air_regions_df)

   region   latitude  longitude
0      강남  37.497940  127.02761
1      서초  37.476500  127.03250
2      강동  37.530100  127.12587
3      송파  37.514560  127.10662
4      강서  37.550950  126.84953
5      양천  37.516560  126.86658
6      도봉  37.668770  127.04744
7      노원  37.655260  127.07712
8     동대문  37.574910  127.03963
9      중랑  37.595080  127.09368
10    기상청  37.565560  126.96691
11     마포  37.566320  126.90131
12    서대문  37.576550  126.93784
13     광진  37.538650  127.08237
14     성북  37.589680  127.01667
15     용산  37.531100  126.97960
16     은평  37.602770  126.93040
17     금천  37.456860  126.89576
18     한강  37.527700  126.99818
19     중구  37.563820  126.99760
20     성동  37.563510  127.03722
21     구로  37.495430  126.88740
22    강북*  37.646995  127.01371
23     남현  37.467690  126.95818
24     관악  37.478080  126.95143
25    영등포  37.526670  126.89636
26    현충원  37.500010  126.97422
   region   latitude  longitude
0     종로구  37.573050  126.97934
1     용산구  37.531100  126.97960
2      중

In [9]:
# ------------------------------------------------------------------
# 나중에 리팩토링할 부분
# ------------------------------------------------------------------

import pandas as pd

# CSV 파일을 불러옵니다.
met_data = pd.read_csv(file_path1, usecols=lambda column: column != '지점', encoding='cp949')

# 각 지점명 별로 일시가 744개가 되어야 정상입니다.
expected_timesteps = 744

# 지점명 별로 그룹화하여 일시의 개수를 셉니다.(같은 지점명, )
grouped = met_data.groupby('지점명')['일시'].count()

# 일시가 744개보다 적은 지점들을 찾습니다. (판다스 시리즈 형태로 리턴)
missing_data = grouped[grouped < expected_timesteps]

# 누락된 지점명과 해당 지점의 누락된 일시를 찾습니다.
missing_rows_info = []

for station in missing_data.index:
    station_data = met_data[met_data['지점명'] == station]

    # 1월 1일 00시부터 1월 31일 23시까지의 일시 범위를 생성합니다.
    full_range = pd.date_range(start='2019-01-01 00:00', end='2019-01-31 23:00', freq='H')

    # 누락된 일시를 찾습니다.
    station_data['일시'] = pd.to_datetime(station_data['일시'], errors='coerce', format='%Y-%m-%d %H:%M')
    missing_dates = full_range.difference(station_data['일시'])

    # 누락된 일시가 있을 경우 정보를 저장합니다.
    for missing_date in missing_dates:
        row_info = {'지점명':station, '일시': missing_date}
        missing_rows_info.append(row_info)

# missing row, met_data의 일시를 datetime으로 변환
for row in missing_rows_info:
  row['일시'] = pd.to_datetime(row['일시'], format='%Y-%m-%d %H:%M:%S')

met_data['일시'] = pd.to_datetime(met_data['일시'], format='%Y-%m-%d %H:%M')

# missing data를 복구.
for missing_row in missing_rows_info:
  location = missing_row['지점명']
  date = missing_row['일시']
  temp_data = met_data[(met_data['지점명']!=location) & (met_data['일시'] == date)]
  tmp = temp_data['기온(°C)'].mean()
  new_row = pd.DataFrame([{'지점명':location, '일시':date, '기온(°C)':tmp}])
  # dataframe에 삽입
  met_data = pd.concat([met_data, new_row], ignore_index = True)


# 정렬
met_data = met_data.sort_values(by=['지점명', '일시']).reset_index(drop=True)
# ------------------------------------------------------------------

<ipython-input-9-b7930eb35bd7>:26: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(start='2019-01-01 00:00', end='2019-01-31 23:00', freq='H')
<ipython-input-9-b7930eb35bd7>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_data['일시'] = pd.to_datetime(station_data['일시'], errors='coerce', format='%Y-%m-%d %H:%M')
<ipython-input-9-b7930eb35bd7>:26: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(start='2019-01-01 00:00', end='2019-01-31 23:00', freq='H')
<ipython-input-9-b7930eb35bd7>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [10]:
# 서울시의 위도와 경도 범위 설정
seoul_lat_min, seoul_lat_max = 37.4133, 37.7151  # 서울시 남북 경계 위도
seoul_lon_min, seoul_lon_max = 126.7341, 127.2693  # 서울시 동서 경계 경도

# 그리드 크기 설정 (32x32)
grid_size = 32

# 각 위도와 경도를 그리드 좌표로 변환하는 함수
def latlon_to_grid(lat, lon, lat_min, lat_max, lon_min, lon_max, grid_size):
    # 위도와 경도를 32x32 그리드 좌표로 변환
    row = int((lat - lat_min) / (lat_max - lat_min) * (grid_size - 1))
    col = int((lon - lon_min) / (lon_max - lon_min) * (grid_size - 1))
    return row, col

# grid에 meteorological data 저장
# 기상 데이터에서 위도와 경도 정보를 사용하여 지역을 그리드로 변환
regions = met_data['지점명'].unique()
grid_data = np.zeros((len(met_data['일시'].unique()), grid_size, grid_size,1))  # 온도, 습도, 풍속용

# 각 지역에 대해 위도, 경도를 그리드 좌표로 변환하고 매핑
for region in regions:
    met_sub_data = met_data[met_data['지점명'] == region] # meteorological data
    #print(met_sub_data)
    region_data = met_regions_df[met_regions_df['region'] == region] # 위도, 경도 정보
    lat = region_data['latitude'].values[0]
    lon = region_data['longitude'].values[0]
    grid_row, grid_col = latlon_to_grid(lat, lon, seoul_lat_min, seoul_lat_max, seoul_lon_min, seoul_lon_max, grid_size)

    # 각 좌표에 해당하는 그리드 위치에 기상 데이터를 할당 (온도, 습도, 풍속 데이터)
    #print(len(met_sub_data['기온(°C)'].values))
    #print(grid_row, grid_col)
    grid_data[:, grid_row, grid_col,0] = met_sub_data['기온(°C)'].values

# 마찬가지로 label grid에 지역별 오염물질의 농도 매핑
label_regions = air_regions_df['region'].unique()
label_grid_data = np.zeros((len(air_quality_data['측정일시'].unique()), grid_size, grid_size,1))

for region in label_regions:
  # lat, lon 저장, row col 뽑아내기, 데이터 저장
  air_sub_data = air_quality_data[air_quality_data['측정소명'] == region]
  lat = air_regions_df[air_regions_df['region'] == region]['latitude']
  lon = air_regions_df[air_regions_df['region']==region]['longitude']

  l_grid_row, l_grid_col = latlon_to_grid(lat, lon, seoul_lat_min, seoul_lat_max, seoul_lon_min, seoul_lon_max, grid_size)

  print(l_grid_row, l_grid_col)
  print(len(air_sub_data['오존(ppm)']))
  label_grid_data[:, l_grid_row, l_grid_col,0] = air_sub_data['오존(ppm)']

16 14
744
12 14
744
15 15
744
19 11
744
16 11
744
15 9
744
12 20
744
24 19
744
18 20
744
26 18
744
18 16
744
15 17
744
24 16
744
16 17
744
10 11
744
11 9
744
14 6
744
6 12
744
10 7
744
8 8
744
4 9
744
8 17
744
6 17
744
11 22
744
10 21
744


<ipython-input-10-bfd591c0328d>:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  row = int((lat - lat_min) / (lat_max - lat_min) * (grid_size - 1))
<ipython-input-10-bfd591c0328d>:12: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  col = int((lon - lon_min) / (lon_max - lon_min) * (grid_size - 1))


In [ ]:
print(label_grid_data[:,16,14,0])

[0.005 0.004 0.002 0.002 0.002 0.002 0.003 0.007 0.002 0.003 0.012 0.028
 0.029 0.025 0.028 0.032 0.032 0.03  0.025 0.02  0.022 0.026 0.028 0.028
 0.026 0.024 0.021 0.018 0.016 0.011 0.004 0.004 0.005 0.012 0.005 0.019
 0.023 0.022 0.022 0.024 0.026 0.02  0.011 0.008 0.005 0.002 0.002 0.003
 0.014 0.018 0.008 0.006 0.002 0.004 0.007 0.003 0.002 0.002 0.004 0.004
 0.009 0.02  0.013 0.012 0.011 0.007 0.003 0.002 0.003 0.003 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.003
 0.003 0.004 0.005 0.007 0.006 0.004 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.003 0.014 0.02  0.027 0.028 0.023 0.021 0.019 0.014 0.016 0.019
 0.02  0.023 0.026 0.03  0.03  0.027 0.022 0.016 0.012 0.013 0.014 0.006
 0.003 0.003 0.002 0.005 0.012 0.014 0.012 0.006 0.004 0.005 0.006 0.009
 0.01  0.016 0.022 0.023 0.023 0.024 0.021 0.019 0.016 0.009 0.003 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.003 0.004
 0.012 0.024 0.019 0.023 0.021 0.022 0.019 0.008 0.

In [ ]:
X = grid_data
Y = label_grid_data

# 모델 생성
model = Sequential([
    Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=(32,32,1)),
    MaxPooling2D(pool_size = (2,2)),
    Conv2D(64, kernel_size = (3,3), activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(32*32, activation = 'linear')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 데이터 변환
Y = Y.reshape(Y.shape[0], -1)

# 모델 학습
model.fit(X, Y, epochs=10, batch_size = 32, validation_split=0.2)

# 모델 평가
loss = model.evaluate(X,Y)
print(f"Loss: {loss}")

# 모델 사용
Ozone_predict = model.predict(X)
Ozone_predict = Ozone_predict.reshape(-1, 32, 32)

print(Ozone_predict)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 0.0026 - val_loss: 1.4406e-05
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 1.0536e-05 - val_loss: 4.9208e-06
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 3.9782e-06 - val_loss: 2.8723e-06
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 2.3295e-06 - val_loss: 2.5930e-06
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 2.4071e-06 - val_loss: 2.5967e-06
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 2.4355e-06 - val_loss: 2.5965e-06
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 2.4031e-06 - val_loss: 2.5622e-06
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 2.2480e-06 - val_loss: 2.5714e-06
Epoch 9/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 2.4056e-06 - val_loss: 2.5965e-06
Epoch 10/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 2.2377e-06 - val_loss: 2.5452e-06
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.1845e-06
Loss: 2.381279